## Load Data

In [ ]:
# call module or whatever

## Preprocess Data

In [ ]:
# Example from homework
# Label encode the y data
y_encoder = LabelEncoder().fit(y)
y = y_encoder.transform(y)

# Convert values to numpy arrays
X = np.array(X)

# Split the training dataset into training and validation sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

## Create CNN Model

In [ ]:
# Import function from cnn_model.py
from cnn_model import create_cnn_model

# Define parameters
input_shape = (128, 128, 3)
num_classes = 2

# Create CNN model
model = create_cnn_model(input_shape, num_classes)

# Display the model summary
model.summary()

## Train Model

In [ ]:
# Example from homework
# Train the model
batch_size = 1000
epochs = 100
history = model.fit(
    X_train, 
    y_train,
    validation_data=(X_test, y_test),
    epochs=epochs
)

## Evaluate Model

In [ ]:
# Plot history
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()